Imports

In [142]:
from kerchunk.tiff import tiff_to_zarr
from kerchunk.combine import MultiZarrToZarr
import ujson
import pystac_client as pc
from rich.console import Console
console = Console()
import geopandas
import xarray as xr
import fsspec
import kerchunk
import zarr
import kerchunk.hdf
import fsspec
import tifffile

Filepaths

In [33]:
path = "/home/otto/s1_zarr/SIG0_20250401T165949__VH_A044_E048N015T3_EU020M_V1M1R2_S1AIWGRDH_TUWIEN.tif"
path2 = "/home/otto/s1_zarr/SIG0_20240215T163540__VH_A175_E051N015T3_EU020M_V1M1R2_S1AIWGRDH_TUWIEN(2).tif"
dest = "/home/otto/s1_zarr/out.json"

STAC Client

In [3]:
client = pc.Client.open("https://stac.eodc.eu/api/v1")
collection = client.get_collection("SENTINEL1_SIG0_20M")

In [61]:
bbox_aut = [9.25, 46.31, 12, 48.18]
time_range = "2024-01-15/2024-01-20"

In [62]:
search = client.search(
    collections=["SENTINEL1_SIG0_20M"],
    bbox=bbox_aut,
    datetime=time_range
    )

In [70]:
items_eodc = search.item_collection()
items_eodc

Creating Zarr metadata json files

In [74]:
# tiff_url = items_eodc[1].assets["VH"].href
# ref = tiff_to_zarr(tiff_url)
# with open("out2.json", "w") as f:
#     json.dump(ref, f)

In [126]:
urls=[]
for item in items_eodc:
    urls.append(item.assets["VH"].href)

In [75]:
# refs = []
# for item in items_eodc:
#     tiff_url = item.assets["VH"].href
#     refs.append(tiff_to_zarr(tiff_url))

Combining single jsons

In [ ]:
with open("out.json") as f1:
    m1 = ujson.load(f1)

with open("out2.json") as f2:
    m2 = ujson.load(f2)

mappings = [m1, m2]

times = []
for m in mappings:
    zattrs = ujson.loads(m[".zattrs"])
    zattrs["_ARRAY_DIMENSIONS"] = ["time", "Y", "X"]
    m[".zattrs"] = ujson.dumps(zattrs)
    times.append(zattrs["sensing_date"])

In [141]:
url

'https://data.eodc.eu/collections/SENTINEL1_SIG0_20M/V1M1R2/EQUI7_EU020M/E048N015T3/SIG0_20240120T165146__VH_A146_E048N015T3_EU020M_V1M1R2_S1AIWGRDH_TUWIEN.tif'

In [140]:
tifffile.tiff2fsspec(url=url)

TypeError: tiff2fsspec() missing 1 required positional argument: 'filename'

In [134]:
mzz = MultiZarrToZarr(
    mappings,
    remote_protocol="https",
    coo_map={"time":times},
    concat_dims=["time"]
)

In [135]:
times

['2024-01-20 16:51:46', '2024-01-20 16:51:21']

In [136]:
combined = mzz.translate()

RuntimeError: asyncio.run() cannot be called from a running event loop

Reading metadata into xarray ds

In [121]:
ds = xr.open_dataset(
    "reference://",
    engine="zarr",
    backend_kwargs={
        "consolidated": False,
        "storage_options": {
            "fo": "combined.json",
            "remote_protocol": "s3",
            "remote_options": {"anon": False},
            "asynchronous":False
        },
    },
)

/home/otto/miniforge3/envs/summerjob/lib/python3.11/site-packages/zarr/storage/_fsspec.py:195: UserWarning: fs (<fsspec.implementations.reference.ReferenceFileSystem object at 0x7f3e94d96e50>) was not created with `asynchronous=True`, this may lead to surprising behavior
  return cls(fs=fs, path=path, read_only=read_only, allowed_exceptions=allowed_exceptions)


ReferenceNotReachable: Reference "y/0" failed to fetch target s3://noaa-goes16/ABI-L2-SSTF/2020/210/00/OR_ABI-L2-SSTF-M6_G16_s20202100000205_e20202100059513_c20202100105456.nc

In [119]:
ds

<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/52)
    _ARRAY_DIMENSIONS:        ['Y', 'X']
    abs_orbit_number:         52193
    border_noise_removed:     True
    creation_date:            2024-01-20 18:51:15
    creator:                  gfm_ops
    dem_source:               /store_frei/eodc_gfm/datapool/copdem/CopDEM30m_...
    ...                       ...
    ProjFalseEastingGeoKey:   5837287.81977
    ProjFalseNorthingGeoKey:  2121415.69617
    ProjCenterLongGeoKey:     24.0
    ProjCenterLatGeoKey:      53.0
    ModelPixelScale:          [20.0, 20.0, 0.0]
    ModelTiepoint:            [0.0, 0.0, 0.0, 4800000.0, 1800000.0, 0.0]